# Data Cleaning - NYC Department of Education Quality Review and Surveys

**Import Packages**

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## (Classification) How does a school's environment affect its overall student achievement rating?

Predict whether a school will not meet (1), approach (2), meet (3), or exceed (4) target

### Read Data

In [2]:
#read data
quality = pd.read_excel('Data/2018_school_quality.xlsx', header=1)
teacher = pd.read_excel('Data/teacher.xlsx')

In [3]:
quality.columns

Index(['DBN', 'School Name', 'School Type', 'Enrollment',
       'Rigorous Instruction Rating', 'Collaborative Teachers Rating',
       'Supportive Environment Rating', 'Effective School Leadership Rating',
       'Strong Family-Community Ties Rating', 'Trust Rating',
       'Student Achievement Rating', 'Rigorous Instruction - Percent Positive',
       'Collaborative Teachers - Percent Positive',
       'Supportive Environment - Percent Positive',
       'Effective School Leadership - Percent Positive',
       'Strong Family-Community Ties - Percent Positive',
       'Trust - Percent Positive',
       'Quality Review - How interesting and challenging is the curriculum?',
       'Quality Review - How effective is the teaching and learning?',
       'Quality Review - How well does the school assess what students are learning?',
       'Quality Review - How clearly are high expectations communicated to students and staff?',
       'Quality Review - How well do teachers work with each oth

In [3]:
# drop unnecessary columns from quality df
quality.drop(columns=['School Type', 'Rigorous Instruction - Percent Positive',
                      'Rigorous Instruction Rating', 'Collaborative Teachers Rating',
                      'Effective School Leadership Rating',
                      'Strong Family-Community Ties Rating', 'Trust Rating',
                      'Collaborative Teachers - Percent Positive',
                      'Supportive Environment - Percent Positive',
                      'Effective School Leadership - Percent Positive',
                      'Strong Family-Community Ties - Percent Positive',
                      'Trust - Percent Positive', 'Quality Review - Dates of Review',
                      'Average Incoming ELA Proficiency (Based on 5th Grade)',
                      'Average Incoming Math Proficiency (Based on 5th Grade)'],
             inplace=True)

In [4]:
quality.columns = ['DBN', 'School Name', 'Enrollment',
                   'Supportive Environment Rating',
                   'Student Achievement Rating',
                   'Interesting and challenging curriculum',
                   'Effective teaching and learning',
                   'Effective school assessment',
                   'Clear communication - high expectations',
                   'Teacher collaboration',
                   'Safety, inclusivity, social-emotional growth',
                   'Resource allocation and management',
                   'Identifying, tracking, and meeting goals',
                   'Thoughtful teacher development and evaluation',
                   'School decision evaluation and adjustment',
                   'Percent English Language Learners',
                   'Percent Students with Disabilities',
                   'Percent Self-Contained', 'Economic Need Index',
                   'Percent in Temp Housing', 'Percent HRA Eligible',
                   'Percent Asian', 'Percent Black',
                   'Percent Hispanic', 'Percent White',
                   'Principal experience at this school',
                   'Percent of teachers with 3+ years of experience',
                   'Student Attendance Rate',
                   'Percent of Students Chronically Absent',
                   'Teacher Attendance Rate']

In [5]:
quality.head()

,DBN,School Name,Enrollment,Supportive Environment Rating,Student Achievement Rating,Interesting and challenging curriculum,Effective teaching and learning,Effective school assessment,Clear communication - high expectations,Teacher collaboration,...,Percent HRA Eligible,Percent Asian,Percent Black,Percent Hispanic,Percent White,Principal experience at this school,Percent of teachers with 3+ years of experience,Student Attendance Rate,Percent of Students Chronically Absent,Teacher Attendance Rate
0,01M015,P.S. 015 Roberto Clemente,173,Exceeding Target,Exceeding Target,Well Developed,Proficient,Proficient,Proficient,Proficient,...,0.786,0.098,0.254,0.601,0.035,7.8,0.625,0.939,0.186,0.985
1,01M019,P.S. 019 Asher Levy,244,Meeting Target,Exceeding Target,Well Developed,Well Developed,Well Developed,Well Developed,Well Developed,...,0.549,0.094,0.197,0.631,0.066,8.8,0.762,0.914,0.353,0.968
2,01M020,P.S. 020 Anna Silver,463,Approaching Target,Approaching Target,Developing,Developing,Developing,Proficient,Developing,...,0.693,0.309,0.102,0.523,0.026,1.8,0.622,0.930,0.274,0.967
3,01M034,P.S. 034 Franklin D. Roosevelt,320,Approaching Target,Approaching Target,Developing,Developing,Developing,Proficient,Proficient,...,0.863,0.034,0.316,0.613,0.031,1.7,0.737,0.901,0.391,0.976
4,01M063,The STAR Academy - P.S.63,177,Meeting Target,Exceeding Target,Well Developed,Proficient,Well Developed,Proficient,Well Developed,...,0.638,0.028,0.175,0.650,0.107,10.9,0.474,0.927,0.261,0.980


In [7]:
teacher.columns

Index(['DBN', 'School Name', 'Total Parent \nResponse Rate',
       'Total Teacher Response Rate', 'Total Student Response Rate',
       'Collaborative Teachers Score', 'Effective School Leadership Score',
       'Rigorous Instruction Score', 'Supportive Environment Score',
       'Strong Family-Community Ties Score', 'Trust Score'],
      dtype='object')

In [6]:
# drop unnecessary columns from teacher df
teacher.drop(teacher.index[0],
             inplace=True)
teacher.drop(columns=['School Name', 'Total Parent \nResponse Rate',
                      'Total Teacher Response Rate',
                      'Total Student Response Rate',
                      'Supportive Environment Score'],
             inplace=True)

In [7]:
teacher.head()

,DBN,Collaborative Teachers Score,Effective School Leadership Score,Rigorous Instruction Score,Strong Family-Community Ties Score,Trust Score
1,01M015,4.12,4.09,3.91,3.80,3.99
2,01M019,4.27,4.25,4.64,4.10,3.55
3,01M020,2.76,3.04,1.94,3.64,3.12
4,01M034,2.40,2.45,2.04,3.33,2.32
5,01M063,4.53,4.07,4.14,4.01,3.95


In [8]:
# merge quality and teacher dfs
nyc_schools = pd.merge(quality, teacher, on='DBN')

In [9]:
nyc_schools.head()

,DBN,School Name,Enrollment,Supportive Environment Rating,Student Achievement Rating,Interesting and challenging curriculum,Effective teaching and learning,Effective school assessment,Clear communication - high expectations,Teacher collaboration,...,Principal experience at this school,Percent of teachers with 3+ years of experience,Student Attendance Rate,Percent of Students Chronically Absent,Teacher Attendance Rate,Collaborative Teachers Score,Effective School Leadership Score,Rigorous Instruction Score,Strong Family-Community Ties Score,Trust Score
0,01M015,P.S. 015 Roberto Clemente,173,Exceeding Target,Exceeding Target,Well Developed,Proficient,Proficient,Proficient,Proficient,...,7.8,0.625,0.939,0.186,0.985,4.12,4.09,3.91,3.80,3.99
1,01M019,P.S. 019 Asher Levy,244,Meeting Target,Exceeding Target,Well Developed,Well Developed,Well Developed,Well Developed,Well Developed,...,8.8,0.762,0.914,0.353,0.968,4.27,4.25,4.64,4.10,3.55
2,01M020,P.S. 020 Anna Silver,463,Approaching Target,Approaching Target,Developing,Developing,Developing,Proficient,Developing,...,1.8,0.622,0.930,0.274,0.967,2.76,3.04,1.94,3.64,3.12
3,01M034,P.S. 034 Franklin D. Roosevelt,320,Approaching Target,Approaching Target,Developing,Developing,Developing,Proficient,Proficient,...,1.7,0.737,0.901,0.391,0.976,2.40,2.45,2.04,3.33,2.32
4,01M063,The STAR Academy - P.S.63,177,Meeting Target,Exceeding Target,Well Developed,Proficient,Well Developed,Proficient,Well Developed,...,10.9,0.474,0.927,0.261,0.980,4.53,4.07,4.14,4.01,3.95


### Data Transformation

In [12]:
nyc_schools['Teacher collaboration'].value_counts()

Proficient         578
Well Developed     432
Developing          90
Under Developed      4
Name: Teacher collaboration, dtype: int64

In [13]:
nyc_schools["Supportive Environment Rating"].value_counts()

Meeting Target        698
Approaching Target    279
Exceeding Target      220
Not Meeting Target     14
Name: Supportive Environment Rating, dtype: int64

In [10]:
# convert target ratings into numbers
for col in nyc_schools.columns[3:5]:
    nyc_schools[col] = nyc_schools[col].map(lambda x:
                                            1 if x == 'Not Meeting Target'
                                            else (2 if x == 'Approaching Target' else
                                                  (3 if x == 'Meeting Target' else 4)))

In [11]:
# function to convert categorical ratings into number ratings
def rating_to_number(df, column):
    df[column] = df[column].map(lambda x:
                                1 if x == 'Under Developed'
                                else (2 if x == 'Developing' else
                                      (3 if x == 'Proficient' else 4)))
    return df[column]

In [12]:
# convert quality review ratings into numbers
for column in nyc_schools.columns[5:15]:
    rating_to_number(nyc_schools, column)

In [13]:
# create column for borough
nyc_schools['Borough'] = nyc_schools['DBN'].map(lambda x:
                                               'Manhattan' if x[2] == 'M'
                                               else ('Bronx' if x[2] == 'X'
                                                    else ('Brooklyn' if x[2] == 'K'
                                                         else 'Queens' if x[2] == 'Q'
                                                          else 'Staten Island')))

In [14]:
# create column for district
nyc_schools['District'] = nyc_schools['DBN'].apply(lambda x:
                                                   841 if x[:3] == '84M'
                                                   else (842 if x[:3] == '84X'
                                                         else (843 if x[:3] == '84K'
                                                               else(844 if x[:3] == '84Q'
                                                                    else (845 if x[:3] == '84R'
                                                                          else x[:2])))))

nyc_schools['District'] = pd.to_numeric(nyc_schools['District'])

### Missing Values

In [19]:
nyc_schools.columns

Index(['DBN', 'School Name', 'Enrollment', 'Supportive Environment Rating',
       'Student Achievement Rating', 'Interesting and challenging curriculum',
       'Effective teaching and learning', 'Effective school assessment',
       'Clear communication - high expectations', 'Teacher collaboration',
       'Safety, inclusivity, social-emotional growth',
       'Resource allocation and management',
       'Identifying, tracking, and meeting goals',
       'Thoughtful teacher development and evaluation',
       'School decision evaluation and adjustment',
       'Percent English Language Learners',
       'Percent Students with Disabilities', 'Percent Self-Contained',
       'Economic Need Index', 'Percent in Temp Housing',
       'Percent HRA Eligible', 'Percent Asian', 'Percent Black',
       'Percent Hispanic', 'Percent White',
       'Principal experience at this school',
       'Percent of teachers with 3+ years of experience',
       'Student Attendance Rate', 'Percent of Student

In [20]:
nyc_schools.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1276 entries, 0 to 1275
Data columns (total 37 columns):
DBN                                                1276 non-null object
School Name                                        1276 non-null object
Enrollment                                         1276 non-null int64
Supportive Environment Rating                      1276 non-null int64
Student Achievement Rating                         1276 non-null int64
Interesting and challenging curriculum             1276 non-null int64
Effective teaching and learning                    1276 non-null int64
Effective school assessment                        1276 non-null int64
Clear communication - high expectations            1276 non-null int64
Teacher collaboration                              1276 non-null int64
Safety, inclusivity, social-emotional growth       1276 non-null int64
Resource allocation and management                 1276 non-null int64
Identifying, tracking, and meeting goal

In [15]:
# percent of missing values for each column
for col in nyc_schools.columns:
    if nyc_schools[col].isna().any() == True:
        per_missing = len(nyc_schools.loc[nyc_schools[col].isna()
                                          == True]
                          )/len(nyc_schools) * 100
        print(f'{col}: {round(per_missing,2)}%')

Principal experience at this school: 13.87%
Percent of teachers with 3+ years of experience: 13.24%
Student Attendance Rate: 2.04%
Percent of Students Chronically Absent: 2.35%
Teacher Attendance Rate: 13.24%
Collaborative Teachers Score: 16.22%
Effective School Leadership Score: 16.22%
Rigorous Instruction Score: 16.3%
Strong Family-Community Ties Score: 16.38%
Trust Score: 4.7%


In [16]:
# replace missing values with median
for col in nyc_schools.columns:
    if nyc_schools[col].isna().any() == True:
        nyc_schools[col].fillna(nyc_schools[col].median(),
                                inplace=True)

In [17]:
nyc_schools.to_csv('Data/nyc_schools.csv', index=False)

## (Regression) How does a school's environment affect its students' ELA Assessment scores?

Predict percentage of students who will receive 3+ on the ELA assessment

In [18]:
nyc_schools = pd.read_csv('Data/nyc_schools.csv')

In [19]:
ela = pd.read_excel("Data/school-ela-results-2018.xlsx",
                    header=None)

In [20]:
# columns to keep from ela df
ela = ela.loc[ela[3] == "All Grades"]
ela = ela.drop(columns=[0, 2, 3, 4, 5, 7, 8, 10, 12, 14, 16])
ela.columns = ["DBN", "Number Tested", "Level 1 %", "Level 2 %",
               "Level 3 %", "Level 4 %", "Level 3+ %"]

In [21]:
# merge df with survey results and df with ela assessment scores
nyc_schools_ela = pd.merge(nyc_schools.drop('Student Achievement Rating',
                                             axis=1),
                            ela, on="DBN")

In [22]:
nyc_schools_ela.head()

,DBN,School Name,Enrollment,Supportive Environment Rating,Interesting and challenging curriculum,Effective teaching and learning,Effective school assessment,Clear communication - high expectations,Teacher collaboration,"Safety, inclusivity, social-emotional growth",...,Strong Family-Community Ties Score,Trust Score,Borough,District,Number Tested,Level 1 %,Level 2 %,Level 3 %,Level 4 %,Level 3+ %
0,01M015,P.S. 015 Roberto Clemente,173,4,4,3,3,3,3,4,...,3.80,3.99,Manhattan,1,67,7.46269,25.3731,46.2687,20.8955,67.1642
1,01M019,P.S. 019 Asher Levy,244,3,4,4,4,4,4,4,...,4.10,3.55,Manhattan,1,92,11.9565,32.6087,39.1304,16.3043,55.4348
2,01M020,P.S. 020 Anna Silver,463,2,2,2,2,3,2,3,...,3.64,3.12,Manhattan,1,209,30.1435,30.1435,28.2297,11.4833,39.7129
3,01M034,P.S. 034 Franklin D. Roosevelt,320,2,2,2,2,3,3,2,...,3.33,2.32,Manhattan,1,214,33.6449,39.7196,18.2243,8.41121,26.6355
4,01M063,The STAR Academy - P.S.63,177,3,4,3,4,3,4,4,...,4.01,3.95,Manhattan,1,64,6.25,35.9375,42.1875,15.625,57.8125


In [23]:
nyc_schools_ela['% Tested'] = nyc_schools_ela['Number Tested'] / nyc_schools_ela['Enrollment']
nyc_schools_ela.drop(['Enrollment', 'Number Tested'], axis=1, inplace=True)

In [24]:
nyc_schools_ela.to_csv('Data/nyc_schools_ela.csv', index=False)

## (Regression) How does a school's environment affect its students' Math Assessment scores?

Predict the percentage of students who will receive a 3+ on the math assessment

In [25]:
math = pd.read_excel("Data/school-math-results-2018.xlsx",
                     header=None)

In [26]:
# columns to keep from math df
math = math.loc[math[3]=="All Grades"]
math = math.drop(columns=[0, 2, 3, 4, 5, 7, 8, 10, 12, 14, 16])
math.columns = ["DBN", "Number Tested", "Level 1 %", "Level 2 %",
                "Level 3 %", "Level 4 %", "Level 3+ %"]

In [27]:
# merge df with survey results and df with ela assessment scores
nyc_schools_math = pd.merge(nyc_schools.drop('Student Achievement Rating',
                                             axis=1),
                            math, on="DBN")

In [28]:
nyc_schools_math.head()

,DBN,School Name,Enrollment,Supportive Environment Rating,Interesting and challenging curriculum,Effective teaching and learning,Effective school assessment,Clear communication - high expectations,Teacher collaboration,"Safety, inclusivity, social-emotional growth",...,Strong Family-Community Ties Score,Trust Score,Borough,District,Number Tested,Level 1 %,Level 2 %,Level 3 %,Level 4 %,Level 3+ %
0,01M015,P.S. 015 Roberto Clemente,173,4,4,3,3,3,3,4,...,3.80,3.99,Manhattan,1,67,11.9403,29.8507,32.8358,25.3731,58.209
1,01M019,P.S. 019 Asher Levy,244,3,4,4,4,4,4,4,...,4.10,3.55,Manhattan,1,89,23.5955,17.9775,29.2135,29.2135,58.427
2,01M020,P.S. 020 Anna Silver,463,2,2,2,2,3,2,3,...,3.64,3.12,Manhattan,1,212,41.5094,23.1132,17.9245,17.4528,35.3774
3,01M034,P.S. 034 Franklin D. Roosevelt,320,2,2,2,2,3,3,2,...,3.33,2.32,Manhattan,1,221,54.7511,29.4118,10.8597,4.97738,15.8371
4,01M063,The STAR Academy - P.S.63,177,3,4,3,4,3,4,4,...,4.01,3.95,Manhattan,1,64,18.75,23.4375,37.5,20.3125,57.8125


In [29]:
nyc_schools_math['% Tested'] = nyc_schools_math['Number Tested'] / nyc_schools_math['Enrollment']
nyc_schools_math.drop(['Enrollment', 'Number Tested'], axis=1, inplace=True)

In [30]:
nyc_schools_math.to_csv('Data/nyc_schools_math.csv', index=False)